In [7]:
# 1) import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from pprint import pprint
import datetime as DT


In [2]:
# 2) Create a path to chrome driver: Make sure to change the path!
executable_path = {'executable_path': 'chromedriver'}
# 3) load the path into browser 4) initialize the browser
browser = Browser('chrome', **executable_path, headless=True)


# Data Extraction via WebScraping

In [919]:
# BESTBUY:
###############################################################
# next_url = "https://www.bestbuy.com/site/windows/windows-laptops/pcmcat748300547900.c?id=pcmcat748300547900"
next_url = 'https://www.bestbuy.com/site/laptop-computers/all-laptops/pcmcat138500050001.c?id=pcmcat138500050001'


page_remaining = True
page_no = 1

laptops = {}
bestbuy_laptops = []

browser.visit(next_url)
time.sleep(1.5)

html = browser.html
soup = bs(html, 'html.parser')


while page_remaining == True:
    try:
        mothers = soup.find_all('div', class_='right-column')
        for mother in mothers:
            title = mother.find('div', class_='sku-title').get_text().strip()
            
            old_price = mother.find('div', class_='priceView-hero-price priceView-customer-price').find('span', class_='sr-only').get_text().strip()
            text_price1 = old_price.replace("$", "")
            text_price2 = text_price1.replace("Your price for this item is ", "")
            nocommas = text_price2.replace(",", "")
            price = float(nocommas)
            
            partial_link = mother.find('div', class_='sku-title').find('a')['href'].strip()
            base_url = "https://www.bestbuy.com/"
            link = base_url + partial_link
            model = mother.find('span', class_='sku-value').get_text().strip()
            sku = soup.find('div', class_='sku-model').findAll('div', class_='sku-attribute-title')[1].find('span', class_='sku-value').get_text().strip()
            title_nospace = title.replace(" ", "")
            title_to_list = title_nospace.split("-")

        dic = {
            'title': title,
            'price': price,
            'link': link,
            'upc': sku,
            'model': model,
            'brand' : title_to_list[0]
        }
        bestbuy_laptops.append(dic)

    except AttributeError:
#         print('error found, skipping...')
        print(f'error found on page ${page_no}, skipping...')
        print(page_no)
        print(title)
        
    
    page_no += 1
#     next_page = f'https://www.bestbuy.com/site/windows/windows-laptops/pcmcat748300547900.c?cp={page_no}&id=pcmcat748300547900'
    next_page = f'https://www.bestbuy.com/site/laptop-computers/all-laptops/pcmcat138500050001.c?cp={page_no}&id=pcmcat138500050001'
    browser.visit(next_page)
    time.sleep(2)

    htmltwo = browser.html
    soup = bs(htmltwo, 'html.parser') 
    
    next_page_avail = soup.find_all('h1')
    for h in next_page_avail:
        message = h.text
        if message == "We’re sorry, something went wrong.":
            page_remaining = False





error found on page $1, skipping...
1
Apple - MacBook Pro - 13" Display with Touch Bar - Intel Core i5 - 8GB Memory - 256GB SSD - Silver
error found on page $2, skipping...
2
Apple - MacBook Pro - 13" Display with Touch Bar - Intel Core i5 - 8GB Memory - 128GB SSD (Latest Model) - Space Gray
error found on page $3, skipping...
3
Microsoft - Surface Pro 6 - 12.3" Touch-Screen - Intel Core i5 - 8GB Memory - 256GB Solid State Drive (Latest Model) - Black
error found on page $4, skipping...
4
Microsoft - Surface Book 2 - 15" Touch-Screen PixelSense™ - 2-in-1 Laptop - Intel Core i7 - 16GB Memory - 256GB SSD - Silver
error found on page $5, skipping...
5
Apple - MacBook Air - 13.3" Retina Display - Intel Core i5 - 8GB Memory - 128GB Flash Storage - Space Gray
error found on page $6, skipping...
6
Microsoft - Surface Laptop 2 - 13.5" Touch-Screen - Intel Core i7 - 8GB Memory - 256GB Solid State Drive (Latest Model) - Cobalt Blue
error found on page $7, skipping...
7
Apple - MacBook 12" Retina

In [888]:
###############################################################
# EXTRACTION WITH TRANSFORMATION:
###############################################################

#FRYS:
###############################################################
# 5) initialize the page parameters and the first site visit:
###############################################################
next_page = "https://www.frys.com/search?cat=-73060&nearbyStoreName=false&pType=pDisplay&rows=20&resultpage=0&start=0&rows=20"
page_remaining = True
page_no = 0
increment = 20
start_no = 0
# 6) make a variable that holds the url we want to scrape data from
laptops = {}

frys_laptops = []

# 7) using the browser function, visit the webpage
browser.visit(next_page)
# 8) stop the scrape process for 1.25 seconds using the time function. This allows the webpage to load in full
time.sleep(1.25)

# 9) create the html variable that allows the browser to visit the HTML elements on the webpage
html = browser.html
# 10) initialize the BeautifulSoup module by telling it to parse html from the HTML elements on the webpage.
soup = bs(html, 'html.parser')


# 11) loop through until no products are found:
###############################################################
while page_remaining == True:
    
    # 12) Data retrieval logic: 
    ###############################################################
    try:
        mothers = soup.find_all('div', class_='togrid')
        for mother in mothers:
            title = mother.find('p', class_='productDescp').get_text().strip()
            
            raw_price = mother.find('label', class_='red_txt').get_text().strip()
            text_price = raw_price.replace("$", "")
            nocommas = text_price.replace(",", "")
            price = float(nocommas)
            
            partial_link = mother.find('p', class_='productDescp').find('a')['href'].strip()
            base_url = "https://www.frys.com/"
            link = base_url + partial_link
            
            results = mother.find_all('div', class_='prodModel')
            for result in results:
                new_list = result.find_all('p', class_='mar-btm')
                for p in new_list:
                    dataitem = p.text.replace(" ", "")
                    di = dataitem.replace('\n', "")
                    dataitem_split = di.split(":")
                    if dataitem_split[0] == "Brand":
                        brand = dataitem_split[1]
                    if dataitem_split[0] == "UPC":
                        upc = dataitem_split[1]
                    if dataitem_split[0] == "Model":
                        model = dataitem_split[1]
                        
            # 13) create a new dictionary that will hold the data that we scrape
            dic = {
                'title': title,
                'price': price,
                'link': link,
                'upc': upc,
                'model': model,
                'brand': brand
            }
            print(dic)
            frys_laptops.append(dic)
            
    except AttributeError:
        # errors out when there are 2 prices in red (price with rebate)
        print("error found. skipping...")
    
    
    # 14) then process the Next Page logic:
    ###############################################################
    page_no += 1
    start_no = start_no + increment
    next_page = f'https://www.frys.com/search?cat=-73060&nearbyStoreName=false&pType=pDisplay&rows=20&resultpage={page_no}&start={start_no}&rows=20'

    browser.visit(next_page)
    time.sleep(1.25)

    html = browser.html
    soup = bs(html, 'html.parser') 
    
    next_page_avail = soup.find_all('p')
    for p in next_page_avail:
        message = p.text
        if message == " No products were found that matched your search":
            page_remaining = False
        


{'title': 'Dell E6230 12.5" Refurbished Laptop With Intel Ci5-3320m Processor, 4GB Memory, 250GB Hard Drive, Windows 10', 'price': 399.0, 'link': 'https://www.frys.com/product/9907916?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '696897763632', 'model': 'E6230i5-3320m', 'brand': 'Dell'}
{'title': 'Lenovo N21 11.6" Refurbished Chromebook With Intel Celeron N2840 Processor, 4GB Memory, 16GB Storage, Chrome OS', 'price': 199.0, 'link': 'https://www.frys.com/product/9914956?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '696897764257', 'model': 'N21CeleronN2840', 'brand': 'Lenovo'}
{'title': 'Lenovo TP X220 12.5" Refurbished Laptop With Intel Ci5-2520M Processor, 4GB Memory, 160GB Hard Drive, Windows 10', 'price': 349.0, 'link': 'https://www.frys.com/product/9831155?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '696897752902', 'model': 'TPX220i5-2520M', 'brand': 'Lenovo'}
{'title': 'HP 8470P 14" Refurbished Laptop with Intel Core i5-3230M Processor 8GB Memory, 320GB Hard Drive,DVD, Windows 10 **90 Days Warranty**', 

{'title': 'Dell G7790-7940GRY-PUS 17.3" Gaming Laptop With New Intel 9th Generation Intel® Core" i7-9750H Processor, 16GB Memory, 1TB Hard Drive, 256GB SSD, RTX2070 8GB Graphics, Windows 10', 'price': 1999.0, 'link': 'https://www.frys.com/product/9896625?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '884116315940', 'model': 'G7790i7-9750H', 'brand': 'Dell'}
{'title': 'Acer A315-41-R3BW 15.6" Laptop With Ryzen 5 2500U Processor, 12GB Memory, 1TB Hard Drive, Windows 10', 'price': 549.0, 'link': 'https://www.frys.com/product/9914776?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '191114906934', 'model': 'A315-BWRyzen52500U', 'brand': 'Acer'}
{'title': 'HP 14-ba253cl 14" Refurbished Touchscreen 2-in-1 Laptop With Intel Ci5-8265U Processor, 8GB Memory, 1TB Hard Drive +16GB Optane Memory, Windows 10', 'price': 574.0, 'link': 'https://www.frys.com/product/9911526?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '193015985872', 'model': '14-ba253cli5-8265U', 'brand': 'HP'}
{'title': 'HP 17-ca0010 17.3" Touchscreen Refurbish

{'title': 'Dell G3579-7009BLK-PUS 15.6" FHD Laptop With Intel Ci7-8750H Processor, 16GB Memory, 512GB SSD, GTX1050Ti 4GB Graphics, Windows 10', 'price': 949.0, 'link': 'https://www.frys.com/product/9922676?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '884116294009', 'model': 'G3579Ci7-8750H', 'brand': 'Dell'}
{'title': 'Lenovo IdeaPad Flex 6 14" Touchscreen 2-in-1 Laptop With Intel Ci7-8550U Processor, 16GB Memory, 512GB SSD, Windows 10', 'price': 1199.0, 'link': 'https://www.frys.com/product/9802395?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '192330716864', 'model': 'IdeapadFlex6', 'brand': 'Lenovo'}
{'title': 'EVOO 13.3" Intel Celeron 2-in-1 Touchscreen Laptop With 4GB Memory, 32GB Hard Drive and Windows 10', 'price': 299.0, 'link': 'https://www.frys.com/product/9696653?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '812550030471', 'model': 'EVL2in1-1332-BK', 'brand': 'EVOO'}
{'title': 'MS Surface 13.5" Platinum Laptop With Intel Ci7 Processor, 8GB Memory, 256GB, Touch, Win10S DAJ-00001', 'price': 899.0, 'l

{'title': 'Dell Insp.15 Ci7-7700HQ 15.6" Screen Size, 16GB, 1TB+128GB GTX 1050 4GB, Win 10', 'price': 799.0, 'link': 'https://www.frys.com/product/9341530?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '884116275152', 'model': 'i5577-7152BLK-PUS', 'brand': 'Dell'}
{'title': 'Lenovo Ideapad 310 AMD A10-9600P 15.6" Laptop With 8GB Memory, 1TB Hard Drive, Windows 10', 'price': 317.57, 'link': 'https://www.frys.com/product/9131978?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '191376077724', 'model': 'Ideapad310-15ABR', 'brand': 'Lenovo'}
{'title': 'Asus GL704GW-PS74 17.3" Gaming Laptop With Intel Ci7-8750H Processor, 16GB Memory, 512GB SSD, RTX2070 8GB Graphics, Windows 10', 'price': 1552.0, 'link': 'https://www.frys.com/product/9798104?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '192876225288', 'model': 'GL704-PS74Ci7-8750H', 'brand': 'Asus'}
{'title': 'Dell G5587-7835BLK-PUS, 15.6" Laptop With Intel® Core" i7 Processor, 16GB Memory,NVIDIA® GeForce® GTX 1050 Ti, 4GB GDDR5 Graphics Card,1TB Hard Drive + 256GB Soli

{'title': 'MSI GF75 THIN 9SC-027 17.3" FHD Gaming Laptop With 9th Generation Intel Ci7-9750H Processor, 16GB Memory, 512GB SSD, GTX1650 4GB Graphics, Windows 10', 'price': 1199.0, 'link': 'https://www.frys.com/product/9897625?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '824142181324', 'model': 'GF759SC-027i7-9750', 'brand': 'MSI'}
{'title': 'Asus UX433FA-DH74 14" Laptop With Intel Ci7-8565U Processor, 16GB Memory, 512GB SSD, Windows 10', 'price': 1199.0, 'link': 'https://www.frys.com/product/9876345?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '192876091821', 'model': 'UX433FADH74i78565U', 'brand': 'Asus'}
{'title': 'Asus UX533FD-DH74 15.6" Laptop With Intel Ci7-8565U Processor, 16GB Memory, 512GB SSD, GTX1050 2GB Graphics, Windows 10', 'price': 1399.0, 'link': 'https://www.frys.com/product/9876335?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '192876114438', 'model': 'UX533FDDH74i7-8565U', 'brand': 'Asus'}
{'title': 'Dell Lat.11 11.6" 3rd Party Refurbished Intel Pentium N3540 Laptop with 4GB Memory, 128GB SS

{'title': 'HP 14-bw065nr E2-9000e 14" Laptop With 4GB Memory, 32eMMC, Windows 10', 'price': 179.0, 'link': 'https://www.frys.com/product/9240609?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '191628043842', 'model': '14-bw065nr', 'brand': 'HP'}
{'title': 'Dell AMD Ryzen 5 2500U 15.6" Touchscreen Laptop With 16GB Memory, 1TB Hard Drive, DVDRW Windows 10', 'price': 599.0, 'link': 'https://www.frys.com/product/9502932?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '884116293347', 'model': 'i5575-A347SLV-PUS', 'brand': 'Dell'}
{'title': 'HP 14-ax050nr 14" Stream Book Purple Laptop With Intel Celeron N3060 Processor, 4GB Memory, 64GB eMMC, Windows 10 - MS Office 365 included', 'price': 199.0, 'link': 'https://www.frys.com/product/9424821?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '192018001930', 'model': '14-ax050nr', 'brand': 'HP'}
{'title': 'Dell i5570-7262SLV-PUS, 15.6" Laptop with Intel® Core" i7 Processor, 12GB Memory, 2TB Hard Drive and Windows 10 Home - Silver', 'price': 599.0, 'link': 'https://www.frys.com/

{'title': 'Acer A315-21-4098, 15.6" Laptop with AMD A4 Processor, 8 GB Memory, 1TB Hard Drive and Windows 10 Home', 'price': 199.0, 'link': 'https://www.frys.com/product/9341850?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '191114445129', 'model': 'A315-21-4098', 'brand': 'Acer'}
{'title': 'HP 10-p020nr IA x5-Z8350 10.1" Screen With 2GB Memory, 32GB eMMC, Touch, 2 in 1, Windows 10', 'price': 149.0, 'link': 'https://www.frys.com/product/9394610?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '190780619896', 'model': '10-p020nr', 'brand': 'HP'}
{'title': 'Dell Inspiron 11, Intel Pentium N3710, 11.6" Touchscreen Convertible Laptop With 4GB Memory, 500GB Hard Drive, Windows 10', 'price': 199.0, 'link': 'https://www.frys.com/product/8940443?site=sr:SEARCH:MAIN_RSLT_PG', 'upc': '884116217930', 'model': 'i3168-3272GRY', 'brand': 'Dell'}
{'title': 'Lenovo Yoga Book 10.1" 2 in 1 with Intel Atom x5-Z8550 Processor, 4GB RAM, 64GB Storage, Android 6.0 - Color Grey', 'price': 199.0, 'link': 'https://www.frys.com/prod

# Data Transformation: Best Buy

In [889]:
bestbuy_df = pd.DataFrame(bestbuy_laptops)
bestbuy_df.head()
# print(len(bestbuy_df))

,brand,link,model,price,title,upc
0,ASUS,https://www.bestbuy.com//site/asus-zenbook-15-...,UX534FTDB77,1399.99,"ASUS - Zenbook 15 15.6"" Gaming Laptop - Intel ...",6346214
1,Dell,https://www.bestbuy.com//site/dell-g3-15-6-gam...,G3579-5731BLK-PUS,637.99,"Dell - G3 15.6"" Gaming Laptop - Intel Core i5 ...",5956013
2,HP,https://www.bestbuy.com//site/hp-probook-14-la...,640 G1-1588,258.99,"HP - ProBook 14"" Laptop - Intel Core i5 - 4GB ...",6293668
3,Lenovo,https://www.bestbuy.com//site/lenovo-legion-y5...,81T20002US,1129.99,"Lenovo - Legion Y545 15.6"" Gaming Laptop - Int...",6212466
4,Dell,https://www.bestbuy.com//site/dell-latitude-15...,1K0YX,824.99,"Dell - Latitude 15.6"" Laptop - Intel Core i5 -...",6311115


In [890]:
bestbuy_df['raw_specs'] = bestbuy_df['title'] + ' - ' + bestbuy_df['upc']

bestbuy_df['raw_specs'].head()

0    ASUS - Zenbook 15 15.6" Gaming Laptop - Intel ...
1    Dell - G3 15.6" Gaming Laptop - Intel Core i5 ...
2    HP - ProBook 14" Laptop - Intel Core i5 - 4GB ...
3    Lenovo - Legion Y545 15.6" Gaming Laptop - Int...
4    Dell - Latitude 15.6" Laptop - Intel Core i5 -...
Name: raw_specs, dtype: object

In [891]:
specs = []

specs = bestbuy_df['raw_specs'].str.split(' - ', expand = True)

specs.head()

,0,1,2,3,4,5,6,7
0,ASUS,"Zenbook 15 15.6"" Gaming Laptop",Intel Core i7,16GB Memory,NVIDIA GeForce GTX 1650,1TB Solid State Drive,"Rose Gold Trim, Dark Royal Blue",6346214
1,Dell,"G3 15.6"" Gaming Laptop",Intel Core i5,8GB Memory,NVIDIA GeForce GTX 1050 Ti,1TB Hard Drive,Black,5956013
2,HP,"ProBook 14"" Laptop",Intel Core i5,4GB Memory,128GB Solid State Drive,Pre-Owned,Black,6293668
3,Lenovo,"Legion Y545 15.6"" Gaming Laptop",Intel Core i7,8GB Memory,NVIDIA GeForce GTX 1650,256GB Solid State Drive,Black,6212466
4,Dell,"Latitude 15.6"" Laptop",Intel Core i5,8GB Memory,256GB Solid State Drive,Black,6311115,None


## Brand

In [892]:
raw_brand_col = []
brand_col = []

raw_brand_col = specs[0]

for x in raw_brand_col:
    new_brand = x.replace('New!', '')
    brand_col.append(new_brand)

brand_col

specs['brand'] = brand_col

specs.head()

,0,1,2,3,4,5,6,7,brand
0,ASUS,"Zenbook 15 15.6"" Gaming Laptop",Intel Core i7,16GB Memory,NVIDIA GeForce GTX 1650,1TB Solid State Drive,"Rose Gold Trim, Dark Royal Blue",6346214,ASUS
1,Dell,"G3 15.6"" Gaming Laptop",Intel Core i5,8GB Memory,NVIDIA GeForce GTX 1050 Ti,1TB Hard Drive,Black,5956013,Dell
2,HP,"ProBook 14"" Laptop",Intel Core i5,4GB Memory,128GB Solid State Drive,Pre-Owned,Black,6293668,HP
3,Lenovo,"Legion Y545 15.6"" Gaming Laptop",Intel Core i7,8GB Memory,NVIDIA GeForce GTX 1650,256GB Solid State Drive,Black,6212466,Lenovo
4,Dell,"Latitude 15.6"" Laptop",Intel Core i5,8GB Memory,256GB Solid State Drive,Black,6311115,None,Dell


## Screen Size

In [893]:
raw_size_col = []
size_col = []

raw_size_col = specs[1]

for x in raw_size_col:
    # find the first instance of "
    new_size1 = x[0:x.find('"')]
    # find the last space before the "
    new_size2 = new_size1[new_size1.rfind(' ')+1:]
    # convert it to a float for the screen size:
    size_col.append(float(new_size2))

size_col

specs['screenSize'] = size_col

specs.head()


,0,1,2,3,4,5,6,7,brand,screenSize
0,ASUS,"Zenbook 15 15.6"" Gaming Laptop",Intel Core i7,16GB Memory,NVIDIA GeForce GTX 1650,1TB Solid State Drive,"Rose Gold Trim, Dark Royal Blue",6346214,ASUS,15.6
1,Dell,"G3 15.6"" Gaming Laptop",Intel Core i5,8GB Memory,NVIDIA GeForce GTX 1050 Ti,1TB Hard Drive,Black,5956013,Dell,15.6
2,HP,"ProBook 14"" Laptop",Intel Core i5,4GB Memory,128GB Solid State Drive,Pre-Owned,Black,6293668,HP,14.0
3,Lenovo,"Legion Y545 15.6"" Gaming Laptop",Intel Core i7,8GB Memory,NVIDIA GeForce GTX 1650,256GB Solid State Drive,Black,6212466,Lenovo,15.6
4,Dell,"Latitude 15.6"" Laptop",Intel Core i5,8GB Memory,256GB Solid State Drive,Black,6311115,None,Dell,15.6


In [894]:
raw_mem_col = []
mem_col = []

raw_mem_col = specs[3]

raw_mem_col
for x in raw_mem_col:
    # find the first instance of 'GB Memory'
    new_mem1 = x[0:x.find('GB Memory')]
    new_mem1 = new_mem1.lstrip()
    try:
        firstchar = int(new_mem1[0])
        mem_col.append(int(new_mem1))
    except TypeError:
        mem_col.append('')
    except ValueError:
        mem_col.append('')
        
# mem_col


In [895]:
# Round 2: for cases when the memory is in the cpu column:
rownum0 = 0
new_mem_col = []

for x in mem_col:
    if x == '':
        new_mem2 = specs.loc[rownum0, 2]
        new_mem2 = new_mem2[0:new_mem2.find('GB Memory')]
        new_mem2 = new_mem2[new_mem2.rfind(' '):]
        new_mem2.lstrip()
        new_mem_col.append(int(new_mem2))
    else:
        new_mem_col.append(x)
    rownum0 +=1

new_mem_col

specs['ram'] = new_mem_col

specs.head()

,0,1,2,3,4,5,6,7,brand,screenSize,ram
0,ASUS,"Zenbook 15 15.6"" Gaming Laptop",Intel Core i7,16GB Memory,NVIDIA GeForce GTX 1650,1TB Solid State Drive,"Rose Gold Trim, Dark Royal Blue",6346214,ASUS,15.6,16
1,Dell,"G3 15.6"" Gaming Laptop",Intel Core i5,8GB Memory,NVIDIA GeForce GTX 1050 Ti,1TB Hard Drive,Black,5956013,Dell,15.6,8
2,HP,"ProBook 14"" Laptop",Intel Core i5,4GB Memory,128GB Solid State Drive,Pre-Owned,Black,6293668,HP,14.0,4
3,Lenovo,"Legion Y545 15.6"" Gaming Laptop",Intel Core i7,8GB Memory,NVIDIA GeForce GTX 1650,256GB Solid State Drive,Black,6212466,Lenovo,15.6,8
4,Dell,"Latitude 15.6"" Laptop",Intel Core i5,8GB Memory,256GB Solid State Drive,Black,6311115,None,Dell,15.6,8


In [896]:
raw_graphics_col = [];
graphics_col = [];

raw_graphics_col = specs[4]

for x in raw_graphics_col:
    try:
        firstchar = int(x[0])
        graphics_col.append('')
    except TypeError:
        graphics_col.append(x)
    except ValueError:
        graphics_col.append(x)
    
graphics_col

specs['graphics'] = graphics_col

specs.head()


,0,1,2,3,4,5,6,7,brand,screenSize,ram,graphics
0,ASUS,"Zenbook 15 15.6"" Gaming Laptop",Intel Core i7,16GB Memory,NVIDIA GeForce GTX 1650,1TB Solid State Drive,"Rose Gold Trim, Dark Royal Blue",6346214,ASUS,15.6,16,NVIDIA GeForce GTX 1650
1,Dell,"G3 15.6"" Gaming Laptop",Intel Core i5,8GB Memory,NVIDIA GeForce GTX 1050 Ti,1TB Hard Drive,Black,5956013,Dell,15.6,8,NVIDIA GeForce GTX 1050 Ti
2,HP,"ProBook 14"" Laptop",Intel Core i5,4GB Memory,128GB Solid State Drive,Pre-Owned,Black,6293668,HP,14.0,4,
3,Lenovo,"Legion Y545 15.6"" Gaming Laptop",Intel Core i7,8GB Memory,NVIDIA GeForce GTX 1650,256GB Solid State Drive,Black,6212466,Lenovo,15.6,8,NVIDIA GeForce GTX 1650
4,Dell,"Latitude 15.6"" Laptop",Intel Core i5,8GB Memory,256GB Solid State Drive,Black,6311115,None,Dell,15.6,8,


In [897]:
HD_col = [];
original_HD_col = specs[5];
rownum = 0;

for x in original_HD_col:
    try:
        firstchar = int(x[0])
        if (x.find('GB') != -1 or x.find('TB') != -1 ):
            HD_col.append(x)
        else:
            HD_col.append(specs.loc[rownum, 4])
    except TypeError:
        HD_col.append(specs.loc[rownum, 4])
    except ValueError:
        HD_col.append(specs.loc[rownum, 4])
    
    rownum +=1

HD_col


specs['hd'] = HD_col

specs.head()

,0,1,2,3,4,5,6,7,brand,screenSize,ram,graphics,hd
0,ASUS,"Zenbook 15 15.6"" Gaming Laptop",Intel Core i7,16GB Memory,NVIDIA GeForce GTX 1650,1TB Solid State Drive,"Rose Gold Trim, Dark Royal Blue",6346214,ASUS,15.6,16,NVIDIA GeForce GTX 1650,1TB Solid State Drive
1,Dell,"G3 15.6"" Gaming Laptop",Intel Core i5,8GB Memory,NVIDIA GeForce GTX 1050 Ti,1TB Hard Drive,Black,5956013,Dell,15.6,8,NVIDIA GeForce GTX 1050 Ti,1TB Hard Drive
2,HP,"ProBook 14"" Laptop",Intel Core i5,4GB Memory,128GB Solid State Drive,Pre-Owned,Black,6293668,HP,14.0,4,,128GB Solid State Drive
3,Lenovo,"Legion Y545 15.6"" Gaming Laptop",Intel Core i7,8GB Memory,NVIDIA GeForce GTX 1650,256GB Solid State Drive,Black,6212466,Lenovo,15.6,8,NVIDIA GeForce GTX 1650,256GB Solid State Drive
4,Dell,"Latitude 15.6"" Laptop",Intel Core i5,8GB Memory,256GB Solid State Drive,Black,6311115,None,Dell,15.6,8,,256GB Solid State Drive


In [898]:
raw_upc_col = [];
upc_col = [];
rownum2 = 0;

raw_upc_col = specs[7];

for x in raw_upc_col:
    try:
        if x is None:
            if specs.loc[rownum2, 6] is None:
                upc_col.append(specs.loc[rownum2, 5])
            else:
                upc_col.append(specs.loc[rownum2, 6])
        else:
            upc_col.append(x)
    except TypeError:
        upc_col.append(specs.loc[rownum2, 6])
    except ValueError:
        upc_col.append(specs.loc[rownum2, 6])
    except KeyError:
        upc_col.append(specs.loc[rownum, 6])
    
    rownum2 +=1
upc_col

specs['upc'] = upc_col

specs.head()

,0,1,2,3,4,5,6,7,brand,screenSize,ram,graphics,hd,upc
0,ASUS,"Zenbook 15 15.6"" Gaming Laptop",Intel Core i7,16GB Memory,NVIDIA GeForce GTX 1650,1TB Solid State Drive,"Rose Gold Trim, Dark Royal Blue",6346214,ASUS,15.6,16,NVIDIA GeForce GTX 1650,1TB Solid State Drive,6346214
1,Dell,"G3 15.6"" Gaming Laptop",Intel Core i5,8GB Memory,NVIDIA GeForce GTX 1050 Ti,1TB Hard Drive,Black,5956013,Dell,15.6,8,NVIDIA GeForce GTX 1050 Ti,1TB Hard Drive,5956013
2,HP,"ProBook 14"" Laptop",Intel Core i5,4GB Memory,128GB Solid State Drive,Pre-Owned,Black,6293668,HP,14.0,4,,128GB Solid State Drive,6293668
3,Lenovo,"Legion Y545 15.6"" Gaming Laptop",Intel Core i7,8GB Memory,NVIDIA GeForce GTX 1650,256GB Solid State Drive,Black,6212466,Lenovo,15.6,8,NVIDIA GeForce GTX 1650,256GB Solid State Drive,6212466
4,Dell,"Latitude 15.6"" Laptop",Intel Core i5,8GB Memory,256GB Solid State Drive,Black,6311115,None,Dell,15.6,8,,256GB Solid State Drive,6311115


In [899]:
new_specs = pd.DataFrame(specs[[2, 'brand', 'screenSize', 'ram', 'graphics', 'hd', 'upc']])

new_specs.rename(columns={2: 'cpu'}, inplace = True)

new_specs.head()


,cpu,brand,screenSize,ram,graphics,hd,upc
0,Intel Core i7,ASUS,15.6,16,NVIDIA GeForce GTX 1650,1TB Solid State Drive,6346214
1,Intel Core i5,Dell,15.6,8,NVIDIA GeForce GTX 1050 Ti,1TB Hard Drive,5956013
2,Intel Core i5,HP,14.0,4,,128GB Solid State Drive,6293668
3,Intel Core i7,Lenovo,15.6,8,NVIDIA GeForce GTX 1650,256GB Solid State Drive,6212466
4,Intel Core i5,Dell,15.6,8,,256GB Solid State Drive,6311115


In [900]:
bestbuy_df.head()
shorter_bestbuy_df = bestbuy_df[['link', 'model', 'price', 'title', 'upc']]
shorter_bestbuy_df.head()


,link,model,price,title,upc
0,https://www.bestbuy.com//site/asus-zenbook-15-...,UX534FTDB77,1399.99,"ASUS - Zenbook 15 15.6"" Gaming Laptop - Intel ...",6346214
1,https://www.bestbuy.com//site/dell-g3-15-6-gam...,G3579-5731BLK-PUS,637.99,"Dell - G3 15.6"" Gaming Laptop - Intel Core i5 ...",5956013
2,https://www.bestbuy.com//site/hp-probook-14-la...,640 G1-1588,258.99,"HP - ProBook 14"" Laptop - Intel Core i5 - 4GB ...",6293668
3,https://www.bestbuy.com//site/lenovo-legion-y5...,81T20002US,1129.99,"Lenovo - Legion Y545 15.6"" Gaming Laptop - Int...",6212466
4,https://www.bestbuy.com//site/dell-latitude-15...,1K0YX,824.99,"Dell - Latitude 15.6"" Laptop - Intel Core i5 -...",6311115


In [901]:
merged1_df = shorter_bestbuy_df.merge(new_specs, on='upc', how='left')

final_bestbuy_df = merged1_df[['brand', 'model', 'screenSize', 'cpu', 'ram', 'hd', 'graphics', 'upc', 'price', 'link', 'title']]


# Add the load date:
now = DT.datetime.now()

final_bestbuy_df['lastUpdated'] = now.strftime("%Y-%m-%d %H:%M")

final_bestbuy_df.head()


C:\Users\Rika\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,brand,model,screenSize,cpu,ram,hd,graphics,upc,price,link,title,lastUpdated
0,ASUS,UX534FTDB77,15.6,Intel Core i7,16,1TB Solid State Drive,NVIDIA GeForce GTX 1650,6346214,1399.99,https://www.bestbuy.com//site/asus-zenbook-15-...,"ASUS - Zenbook 15 15.6"" Gaming Laptop - Intel ...",2019-09-24 17:48
1,Dell,G3579-5731BLK-PUS,15.6,Intel Core i5,8,1TB Hard Drive,NVIDIA GeForce GTX 1050 Ti,5956013,637.99,https://www.bestbuy.com//site/dell-g3-15-6-gam...,"Dell - G3 15.6"" Gaming Laptop - Intel Core i5 ...",2019-09-24 17:48
2,HP,640 G1-1588,14.0,Intel Core i5,4,128GB Solid State Drive,,6293668,258.99,https://www.bestbuy.com//site/hp-probook-14-la...,"HP - ProBook 14"" Laptop - Intel Core i5 - 4GB ...",2019-09-24 17:48
3,Lenovo,81T20002US,15.6,Intel Core i7,8,256GB Solid State Drive,NVIDIA GeForce GTX 1650,6212466,1129.99,https://www.bestbuy.com//site/lenovo-legion-y5...,"Lenovo - Legion Y545 15.6"" Gaming Laptop - Int...",2019-09-24 17:48
4,Dell,1K0YX,15.6,Intel Core i5,8,256GB Solid State Drive,,6311115,824.99,https://www.bestbuy.com//site/dell-latitude-15...,"Dell - Latitude 15.6"" Laptop - Intel Core i5 -...",2019-09-24 17:48


# Data Transformation: Frys

In [902]:
# create a dataframe from the list of dictionaries:
frys_df = pd.DataFrame(frys_laptops)
frys_df
# print(len(frys_df))


,brand,link,model,price,title,upc
0,Dell,https://www.frys.com/product/9907916?site=sr:S...,E6230i5-3320m,399.00,"Dell E6230 12.5"" Refurbished Laptop With Intel...",696897763632
1,Lenovo,https://www.frys.com/product/9914956?site=sr:S...,N21CeleronN2840,199.00,"Lenovo N21 11.6"" Refurbished Chromebook With I...",696897764257
2,Lenovo,https://www.frys.com/product/9831155?site=sr:S...,TPX220i5-2520M,349.00,"Lenovo TP X220 12.5"" Refurbished Laptop With I...",696897752902
3,HP,https://www.frys.com/product/9647583?site=sr:S...,8470PCi5-3230M,399.00,"HP 8470P 14"" Refurbished Laptop with Intel Cor...",696897734717
4,Lenovo,https://www.frys.com/product/9914966?site=sr:S...,TPX240i5-4300U,399.00,"Lenovo TP X240 12.5"" Refurbished Laptop With I...",696897764264
5,Dell,https://www.frys.com/product/9939176?site=sr:S...,E6440Ci5-4200M,599.00,"Dell E6440 14"" Refurbished Laptop With Intel C...",724962745983
6,Lenovo,https://www.frys.com/product/9721554?site=sr:S...,T420Ci5-2520M,349.00,"Lenovo T420 14.1"" Refurbished Laptop With Ci5-...",696897743108
7,Dell,https://www.frys.com/product/9799944?site=sr:S...,E7440Ci5-4200U,499.00,"Dell E7440 14"" Touchscreen Refurbished Laptop ...",696897751165
8,Lenovo,https://www.frys.com/product/9797874?site=sr:S...,X131EIntel1007U,249.00,"Lenovo X131E 11.6"" Refurbished Laptop With Int...",696897751226
9,Lenovo,https://www.frys.com/product/9779764?site=sr:S...,TPT430i5-3210M,449.00,"Lenovo Thinkpad T430 14"" Refurbished Laptop Wi...",696897750199


In [903]:
frys_df['modified_title'] = frys_df['title'].str.replace('"', '",')
frys_df['modified_title'].head()

0    Dell E6230 12.5", Refurbished Laptop With Inte...
1    Lenovo N21 11.6", Refurbished Chromebook With ...
2    Lenovo TP X220 12.5", Refurbished Laptop With ...
3    HP 8470P 14", Refurbished Laptop with Intel Co...
4    Lenovo TP X240 12.5", Refurbished Laptop With ...
Name: modified_title, dtype: object

In [904]:
# for easier parsing:

frownum1 = 0

for x in frys_df['modified_title']:
    new_title = x[x.find(' '):]
    new_title = new_title[new_title.find(' '):]
    new_title = new_title[new_title.find(' '):]
    new_title = new_title.replace('Refurbished ', '')
    new_title = new_title.replace('', '')
    new_title = new_title.replace('-inch', '",')
    new_title = new_title.replace(' Measured Diagonal', '",')
    
    frys_df.loc[frownum1, 'modified_title'] = new_title
        
    frownum1 +=1

frys_df['modified_title']


0       E6230 12.5", Laptop With Intel Ci5-3320m Proc...
1       N21 11.6", Chromebook With Intel Celeron N284...
2       TP X220 12.5", Laptop With Intel Ci5-2520M Pr...
3       8470P 14", Laptop with Intel Core i5-3230M Pr...
4       TP X240 12.5", Laptop With Intel Ci5-4300u Pr...
5       E6440 14", Laptop With Intel Ci5-4200M Proces...
6       T420 14.1", Laptop With Ci5-2520M Processor, ...
7       E7440 14", Touchscreen Laptop With Intel Ci5-...
8       X131E 11.6", Laptop With Intel Celeron 1007U ...
9       Thinkpad T430 14", Laptop With Intel Ci5-3210...
10      EB 8440P 14", Laptop With Intel Ci5-520M Proc...
11      E5440 14", Laptop With Intel Ci5-4300U Proces...
12      G4 11.6", Chromebook With Intel Celeron N2840...
13      TP 11e 11.6", Chromebook With Intel Celeorn N...
14      TP T450 14", Laptop With Intel Ci5-5200U Proc...
15      9470 14", Laptop With Intel Ci5-3427U Process...
16      i5575-A217SLV-PUS 15.6", FHD Laptop With AMD ...
17      TP T440P 14", Laptop Wi

In [905]:
# UPC FIRST because the rest is such a mess:
frys_df['raw_specs'] = frys_df['upc'] + ', ' + frys_df['modified_title']
frys_df.head()

,brand,link,model,price,title,upc,modified_title,raw_specs
0,Dell,https://www.frys.com/product/9907916?site=sr:S...,E6230i5-3320m,399.0,"Dell E6230 12.5"" Refurbished Laptop With Intel...",696897763632,"E6230 12.5"", Laptop With Intel Ci5-3320m Proc...","696897763632, E6230 12.5"", Laptop With Intel ..."
1,Lenovo,https://www.frys.com/product/9914956?site=sr:S...,N21CeleronN2840,199.0,"Lenovo N21 11.6"" Refurbished Chromebook With I...",696897764257,"N21 11.6"", Chromebook With Intel Celeron N284...","696897764257, N21 11.6"", Chromebook With Inte..."
2,Lenovo,https://www.frys.com/product/9831155?site=sr:S...,TPX220i5-2520M,349.0,"Lenovo TP X220 12.5"" Refurbished Laptop With I...",696897752902,"TP X220 12.5"", Laptop With Intel Ci5-2520M Pr...","696897752902, TP X220 12.5"", Laptop With Inte..."
3,HP,https://www.frys.com/product/9647583?site=sr:S...,8470PCi5-3230M,399.0,"HP 8470P 14"" Refurbished Laptop with Intel Cor...",696897734717,"8470P 14"", Laptop with Intel Core i5-3230M Pr...","696897734717, 8470P 14"", Laptop with Intel Co..."
4,Lenovo,https://www.frys.com/product/9914966?site=sr:S...,TPX240i5-4300U,399.0,"Lenovo TP X240 12.5"" Refurbished Laptop With I...",696897764264,"TP X240 12.5"", Laptop With Intel Ci5-4300u Pr...","696897764264, TP X240 12.5"", Laptop With Inte..."


In [906]:
frys_specs = []
frys_specs = frys_df['raw_specs'].str.split(', ', expand = True)
frys_specs.head()
frys_specs[160:]

,0,1,2,3,4,5,6,7,8
160,888631120300,Miix 2,Intel Core i5-4202Y,"11.6""",Touch 2-in1 Laptop With 4GB Memory,128GB SSD,Windows 8.1,None,None
161,193015986077,"17-ca0009 17.3""",Touchscreen Laptop With AMD A9-9425 Processor,8GB Memory,2TB Hard Drive,Windows 10,None,None,None
162,889842170597,"Surface 13.5""","laptop with Intel Core i5 Processor,8GB",256GB,"Touch,Windows 10S DAG-00001-Platinum Color",None,None,None,None
163,190576533580,"Ideapad 310 Intel Core i7-6500U 15.6""",Laptop With 12GB Memory,1TB Hard Drive,Windows 10,None,None,None,None
164,191376122813,"Idea720S 14""",Laptop With Intel Ci7-7500U Processor,16GB Memory,512GB SSD,Windows 10 Geforce 940MX 2GB,None,None,None
165,889349815861,"UX430UA 14""",Laptop With Intel Ci7-8550U Processor,16GB Memory,512GB SSD,Windows 10,None,None,None
166,824142181324,"GF75 THIN 9SC-027 17.3""",FHD Gaming Laptop With 9th Generation Intel Ci...,16GB Memory,512GB SSD,GTX1650 4GB Graphics,Windows 10,None,None
167,192876091821,"UX433FA-DH74 14""",Laptop With Intel Ci7-8565U Processor,16GB Memory,512GB SSD,Windows 10,None,None,None
168,192876114438,"UX533FD-DH74 15.6""",Laptop With Intel Ci7-8565U Processor,16GB Memory,512GB SSD,GTX1050 2GB Graphics,Windows 10,None,None
169,696897735721,"Lat.11 11.6""",3rd Party Intel Pentium N3540 Laptop with 4GB ...,128GB SSD,Windows 10 **90 Days Warranty**,None,None,None,None


## Screen Size

In [907]:
# first found in col 1:
fraw_size = []
f_size = []

fraw_size1 = frys_specs[1]

for x in fraw_size1:
    if (x.find('"') > 0):
        # find the first instance of "
        fsize1 = x[0:x.find('"')]
        # find the last space before the "
        fsize2 = fsize1[fsize1.rfind(' ')+1:]
        try:
            firstchar = int(fsize2[0])
            f_size.append(float(fsize2))
        except:
            f_size.append('')
    else:
        f_size.append('')

# len(f_size)
# f_size

In [908]:
# round 2: found in col 2:
f_size2 = []
frownum2 = 0

for x in f_size:
    if (x == '' or x is None):
        new_size = frys_specs.loc[frownum2, 2]
        new_size2 = new_size.lstrip()
        if (new_size2.find('"') > 0):
            try:
                firstchar = int(new_size[0])
                new_size = new_size[0:new_size.find('"')]
                new_size = new_size[new_size.rfind(' ')+1:]
                f_size2.append(float(new_size))
            except:
                f_size2.append(x)
        else:
            f_size2.append(x)
    else:
        f_size2.append(x)
    
    frownum2 += 1
       
f_size2
    



[12.5,
 11.6,
 12.5,
 14.0,
 12.5,
 14.0,
 14.1,
 14.0,
 11.6,
 14.0,
 14.0,
 14.0,
 11.6,
 11.6,
 14.0,
 14.0,
 15.6,
 14.0,
 12.5,
 14.0,
 14.0,
 17.3,
 15.6,
 15.6,
 15.6,
 15.6,
 15.6,
 11.6,
 15.6,
 15.6,
 15.6,
 15.6,
 15.6,
 14.0,
 14.1,
 14.0,
 11.6,
 17.3,
 15.6,
 15.6,
 17.3,
 15.6,
 14.0,
 17.3,
 14.0,
 14.0,
 15.6,
 14.0,
 14.0,
 15.6,
 15.6,
 14.0,
 13.3,
 13.3,
 15.6,
 15.6,
 15.6,
 14.0,
 13.3,
 14.0,
 15.6,
 15.6,
 11.6,
 15.6,
 14.0,
 11.6,
 15.6,
 15.6,
 15.6,
 11.6,
 15.6,
 17.3,
 11.6,
 11.6,
 12.2,
 14.0,
 15.6,
 15.6,
 15.6,
 15.6,
 15.6,
 14.0,
 13.3,
 13.5,
 15.6,
 15.6,
 11.6,
 11.6,
 15.6,
 15.6,
 15.6,
 13.3,
 13.3,
 15.6,
 14.0,
 15.6,
 14.0,
 13.3,
 8.0,
 15.6,
 14.0,
 17.3,
 15.6,
 15.6,
 17.3,
 13.3,
 11.6,
 12.5,
 15.6,
 15.6,
 14.0,
 15.6,
 15.6,
 17.3,
 15.6,
 11.6,
 13.3,
 '',
 12.5,
 11.6,
 15.6,
 12.5,
 15.6,
 12.3,
 15.6,
 11.6,
 15.6,
 15.6,
 17.3,
 15.6,
 '',
 15.6,
 14.0,
 15.6,
 15.6,
 17.3,
 15.6,
 17.3,
 15.6,
 15.6,
 14.0,
 15.6,
 15.6,
 15.

In [909]:
# round 3: lastly found in col 3:
frys_size = []
frownum2 = 0

for x in f_size2:
    if (x == '' or x is None):
        new_size = frys_specs.loc[frownum2, 3]
        new_size2 = new_size.lstrip()
        if (new_size2.find('"') > 0):
            try:
                firstchar = int(new_size[0])
                new_size = new_size[0:new_size.find('"')]
                new_size = new_size[new_size.rfind(' ')+1:]
                frys_size.append(float(new_size))
            except:
                frys_size.append(x)
        else:
            frys_size.append(x)
    else:
        frys_size.append(x)
    
    frownum2 += 1
        
    

frys_size
    

frys_specs['screenSize'] = frys_size

frys_specs


,0,1,2,3,4,5,6,7,8,screenSize
0,696897763632,"E6230 12.5""",Laptop With Intel Ci5-3320m Processor,4GB Memory,250GB Hard Drive,Windows 10,None,None,None,12.5
1,696897764257,"N21 11.6""",Chromebook With Intel Celeron N2840 Processor,4GB Memory,16GB Storage,Chrome OS,None,None,None,11.6
2,696897752902,"TP X220 12.5""",Laptop With Intel Ci5-2520M Processor,4GB Memory,160GB Hard Drive,Windows 10,None,None,None,12.5
3,696897734717,"8470P 14""",Laptop with Intel Core i5-3230M Processor 8GB ...,"320GB Hard Drive,DVD",Windows 10 **90 Days Warranty**,None,None,None,None,14
4,696897764264,"TP X240 12.5""",Laptop With Intel Ci5-4300u Processor,8GB Memory,500GB Hard Drive,Windows 10,None,None,None,12.5
5,724962745983,"E6440 14""",Laptop With Intel Ci5-4200M Processor,16GB Memory,512GB SSD,Windows 10,None,None,None,14
6,696897743108,"T420 14.1""",Laptop With Ci5-2520M Processor,4GB Memory,250GB Hard Drive and Windows 10,None,None,None,None,14.1
7,696897751165,"E7440 14""",Touchscreen Laptop With Intel Ci5-4200U Processor,8GB Memory,128GB SSD,Windows 10,None,None,None,14
8,696897751226,"X131E 11.6""",Laptop With Intel Celeron 1007U Processor,4GB Memory,320GB Hard Drive,Windows 10 Home,None,None,None,11.6
9,696897750199,"Thinkpad T430 14""",Laptop With Intel Ci5-3210M Processor,8GB Memory,240GB SSD,Windows 10 Pro,None,None,None,14


## CPU

In [910]:
# cpu round 1: col 1:
fraw_cpu = []
fcpu = []

fraw_cpu = frys_specs[1]

for x in fraw_cpu:
    # remove 'processor':
    f_cpu = x.replace(' Processor', '')
    
    # get rid of the screensize:
    if f_cpu.find('"') > 0:
        f_cpu = f_cpu[0:f_cpu.rfind(' ')]
       
    
    
    if f_cpu.find('Intel') > 0:
        f_cpu = f_cpu[f_cpu.find('Intel'):]
        fcpu.append(f_cpu)

    elif f_cpu.find('AMD') > 0:
        f_cpu = f_cpu[f_cpu.find('AMD'):]
        fcpu.append(f_cpu)
    
    else:
        fcpu.append('')

fcpu
        


['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Intel Celeron',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Intel Core i7-8750H',
 'Intel QC Purple',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'AMD A10-9600P',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Intel QC',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Intel Core i7-6500U',
 '',
 '',
 '',
 '',
 '',
 '',
 'Intel Core i7-6700HQ',
 'AMD A12-9700P',
 '',
 '',
 'Intel Co

In [911]:
# cpu round 2:
fcpu1 = []
frnum = 0

# for x from the first round:
for x in fcpu:
    if x == '':
        f_cpu1 = frys_specs.loc[frnum, 2]
        
        # if it says 'Processor' in it:
        if f_cpu1.find('Processor') > 0:
            # cpu info ends at the first instance of 'Processor':
            f_cpu1 = f_cpu1[0:f_cpu1.find(' Processor')]
            
            # cpu info should start after 'With'
            f_cpu1 = f_cpu1.replace('With', 'with')
            f_cpu1 = f_cpu1[f_cpu1.find('with'):]

            # clean the rest out:
            f_cpu2 = f_cpu1.replace('with', '')
            f_cpu2 = f_cpu2.replace('2GB Memor', '')
            f_cpu2 = f_cpu2.replace('4GB Memor', '')
            f_cpu2 = f_cpu2.replace('8GB Memor', '')  
            f_cpu2 = f_cpu2.replace('16GB Memor', '')
            f_cpu2 = f_cpu2[1:]

            if len(f_cpu2) > 2:
                fcpu1.append(f_cpu2)
            else:
                fcpu1.append('')
        
        # if 'Processor' is not part of its name:
        elif (f_cpu1.find('Intel') >= 0) or (f_cpu1.find('AMD') >= 0):
            
            # clean out junk:
            f_cpu2 = f_cpu1.replace('Full HD IPS Laptop 8GB 1TB HDD Windows 10 Home ', '')
            f_cpu2 = f_cpu2.replace(' Laptop with 4GB Memory', '')
            f_cpu2 = f_cpu2.replace('"', '')
            f_cpu2 = f_cpu2.replace(' 4GB Memory', '')
            f_cpu2 = f_cpu2.replace(' Halo Keyboard', '')
            f_cpu2 = f_cpu2.replace(' 8GB 512GB SSD Touchscreen Laptop - Grey', '')
            f_cpu2 = f_cpu2.replace('   - Aluminum', '')
            f_cpu2 = f_cpu2.replace('With', 'with')
            
            
            if f_cpu2.find('with') > 0:
                # then retrieve everything after as cpu:
                f_cpu2 = f_cpu2[f_cpu2.find('with')+5:]
                fcpu1.append(f_cpu2)
                
            elif f_cpu2.find('3rd Party Intel'):
                fcpu1.append(f_cpu2)
                
            elif f_cpu2.find('Intel') >= 0:
                f_cpu2 = f_cpu2[f_cpu2.find('Intel'):]
                fcpu1.append(f_cpu2)

            elif f_cpu2.find('AMD') >= 0:
                f_cpu2 = f_cpu2[f_cpu2.find('AMD'):]
                fcpu1.append(f_cpu2)
            
            else:
                fcpu1.append('')
        else:
            fcpu1.append('')

    else:  # x != '':
        fcpu1.append(x)

        
    frnum += 1
    


fcpu1


['Intel Ci5-3320m',
 'Intel Celeron N2840',
 'Intel Ci5-2520M',
 'Intel Core i5-3230M',
 'Intel Ci5-4300u',
 'Intel Ci5-4200M',
 'Ci5-2520M',
 'Intel Ci5-4200U',
 'Intel Celeron 1007U',
 'Intel Ci5-3210M',
 'Intel Ci5-520M',
 'Intel Ci5-4300U',
 'Intel Celeron N2840',
 'Intel Celeorn N2920',
 'Intel Ci5-5200U',
 'Intel Ci5-3427U',
 'AMD Ryzen 5 2500U',
 'Intel Ci5-4300M',
 'Intel Ci5-3320M',
 'Intel Ci5-3210M',
 'AMD Ryzen5 3500U',
 'Intel Ci7-8550U',
 'Intel Ci5-8250U',
 'AMD Ryzen R3 2200U',
 'Intel Ci5-8250U',
 'AMD Ryzen 5 2500U',
 'Intel Ci5-8250U',
 'Intel Celeron N2930',
 'Ryzen 3 2200U',
 'AMD A6-9225',
 'Ryzen 5 2500U',
 'AMD Ryzen3 2200U',
 'AMD Ryzen5 2500U',
 'Intel Ci5-2520M',
 'Intel Ci5-520M',
 'AMD Ryzen 3 3200U',
 'Intel Celeron N2940',
 'Intel Ci5-8250U',
 'Intel Ci5-8250U',
 'Intel Ci5-8250U',
 'New Intel 9th Generation Intel® Core',
 'Ryzen 5 2500U',
 'Intel Ci5-8265U',
 'AMD A9-9425',
 'Intel Ci7-8565U',
 'Intel Ci5-4300u',
 'Intel Ci7-8565U',
 'Intel Ci5-4300U',
 

In [912]:
# cpu round 3: look in col 3:
fraw_cpu2 = []
fcpu2 = []
frownum3 = 0


for x in fcpu1:
    if x == '':
        # check for nulls:
        if frys_specs.loc[frownum3, 3] is None:
            fcpu2.append('')

        else:    
            # pull the data from col 3:
            f_cpu3 = frys_specs.loc[frownum3, 3]

            # clean the junk out:
            f_cpu4 = f_cpu3.replace('1GB Memory', '')
            f_cpu4 = f_cpu3.replace('2GB Memor', '')
            f_cpu4 = f_cpu4.replace('4GB Memor', '')
            f_cpu4 = f_cpu4.replace('8GB Memor', '')  
            f_cpu4 = f_cpu4.replace('16GB Memor', '')
            f_cpu4 = f_cpu4.replace('320GB', '')
            f_cpu4 = f_cpu4.replace('256GB', '')
            f_cpu4 = f_cpu4.replace('16GB', '')
            f_cpu4 = f_cpu4.replace('4GB', '')
            f_cpu4 = f_cpu4.replace('1TB', '')
            f_cpu4 = f_cpu4.replace('Hard Drive', '')
            f_cpu4 = f_cpu4.replace('Windows 10', '')
            f_cpu4 = f_cpu4.replace('with', 'With')

            f_cpu4 = f_cpu4[f_cpu4.find('With')+4:]
            f_cpu4 = f_cpu4.replace(' Processor', '')
            f_cpu4 = f_cpu4.replace('y,', '')
            f_cpu4 = f_cpu4.replace('Storage', '')
            f_cpu4 = f_cpu4.replace('GB', '')
            f_cpu4 = f_cpu4.replace('"', '')

            if (f_cpu4.find('Intel') > 0) or (f_cpu4.find('AMD') > 0):
                fcpu2.append(f_cpu4[1:])
            else:
                fcpu2.append('')
                
    else:
        fcpu2.append(x)
    
    frownum3 += 1


fcpu2
    
    
  
frys_specs['cpu'] = fcpu2

frys_specs  
    
    

,0,1,2,3,4,5,6,7,8,screenSize,cpu
0,696897763632,"E6230 12.5""",Laptop With Intel Ci5-3320m Processor,4GB Memory,250GB Hard Drive,Windows 10,None,None,None,12.5,Intel Ci5-3320m
1,696897764257,"N21 11.6""",Chromebook With Intel Celeron N2840 Processor,4GB Memory,16GB Storage,Chrome OS,None,None,None,11.6,Intel Celeron N2840
2,696897752902,"TP X220 12.5""",Laptop With Intel Ci5-2520M Processor,4GB Memory,160GB Hard Drive,Windows 10,None,None,None,12.5,Intel Ci5-2520M
3,696897734717,"8470P 14""",Laptop with Intel Core i5-3230M Processor 8GB ...,"320GB Hard Drive,DVD",Windows 10 **90 Days Warranty**,None,None,None,None,14,Intel Core i5-3230M
4,696897764264,"TP X240 12.5""",Laptop With Intel Ci5-4300u Processor,8GB Memory,500GB Hard Drive,Windows 10,None,None,None,12.5,Intel Ci5-4300u
5,724962745983,"E6440 14""",Laptop With Intel Ci5-4200M Processor,16GB Memory,512GB SSD,Windows 10,None,None,None,14,Intel Ci5-4200M
6,696897743108,"T420 14.1""",Laptop With Ci5-2520M Processor,4GB Memory,250GB Hard Drive and Windows 10,None,None,None,None,14.1,Ci5-2520M
7,696897751165,"E7440 14""",Touchscreen Laptop With Intel Ci5-4200U Processor,8GB Memory,128GB SSD,Windows 10,None,None,None,14,Intel Ci5-4200U
8,696897751226,"X131E 11.6""",Laptop With Intel Celeron 1007U Processor,4GB Memory,320GB Hard Drive,Windows 10 Home,None,None,None,11.6,Intel Celeron 1007U
9,696897750199,"Thinkpad T430 14""",Laptop With Intel Ci5-3210M Processor,8GB Memory,240GB SSD,Windows 10 Pro,None,None,None,14,Intel Ci5-3210M


## RAM

In [913]:
ram_string_arr  = frys_df['modified_title']

ram_num_arr = []

for x in ram_string_arr:
    raw_mem = x.replace('memory', 'Memory')
    raw_mem = raw_mem.replace(',', ' ')
    
    if raw_mem.find('Memory') >=0:
        #clean the data:
        ramNo = raw_mem[raw_mem.find('Memory')-5:raw_mem.find('Memory')-3]
        ramNo = ramNo.replace(' ','')
        ram_num_arr.append(int(ramNo))
    
    elif raw_mem.find('8 GB DDR4 SDRAM') >= 0:
        ramNo = 8
        ram_num_arr.append(int(ramNo))
    
    elif raw_mem.find('RAM') >= 0:
        #clean the data:
        ramNo = raw_mem[raw_mem.find('RAM')-4:raw_mem.find('RAM')-3]
        ram_num_arr.append(int(ramNo))
    
    elif raw_mem.find('Full HD IPS Laptop 8GB') >= 0:
        ramNo = 8
        ram_num_arr.append(int(ramNo))
    
    else:
        ram_num_arr.append('')

ram_num_arr

# ram_string_arr[237]
  
frys_specs['ram'] = ram_num_arr

frys_specs  


,0,1,2,3,4,5,6,7,8,screenSize,cpu,ram
0,696897763632,"E6230 12.5""",Laptop With Intel Ci5-3320m Processor,4GB Memory,250GB Hard Drive,Windows 10,None,None,None,12.5,Intel Ci5-3320m,4
1,696897764257,"N21 11.6""",Chromebook With Intel Celeron N2840 Processor,4GB Memory,16GB Storage,Chrome OS,None,None,None,11.6,Intel Celeron N2840,4
2,696897752902,"TP X220 12.5""",Laptop With Intel Ci5-2520M Processor,4GB Memory,160GB Hard Drive,Windows 10,None,None,None,12.5,Intel Ci5-2520M,4
3,696897734717,"8470P 14""",Laptop with Intel Core i5-3230M Processor 8GB ...,"320GB Hard Drive,DVD",Windows 10 **90 Days Warranty**,None,None,None,None,14,Intel Core i5-3230M,8
4,696897764264,"TP X240 12.5""",Laptop With Intel Ci5-4300u Processor,8GB Memory,500GB Hard Drive,Windows 10,None,None,None,12.5,Intel Ci5-4300u,8
5,724962745983,"E6440 14""",Laptop With Intel Ci5-4200M Processor,16GB Memory,512GB SSD,Windows 10,None,None,None,14,Intel Ci5-4200M,16
6,696897743108,"T420 14.1""",Laptop With Ci5-2520M Processor,4GB Memory,250GB Hard Drive and Windows 10,None,None,None,None,14.1,Ci5-2520M,4
7,696897751165,"E7440 14""",Touchscreen Laptop With Intel Ci5-4200U Processor,8GB Memory,128GB SSD,Windows 10,None,None,None,14,Intel Ci5-4200U,8
8,696897751226,"X131E 11.6""",Laptop With Intel Celeron 1007U Processor,4GB Memory,320GB Hard Drive,Windows 10 Home,None,None,None,11.6,Intel Celeron 1007U,4
9,696897750199,"Thinkpad T430 14""",Laptop With Intel Ci5-3210M Processor,8GB Memory,240GB SSD,Windows 10 Pro,None,None,None,14,Intel Ci5-3210M,8


## HD

In [914]:
raw_frys_hd_arr = frys_df['modified_title']

frys_hd_arr = []

for x in raw_frys_hd_arr:
    if x.find('Hard Drive') >= 0:
        # clean data:
        hd = x[x.find('Hard Drive')-6: x.find('Hard Drive')-1]
        hd = hd.replace(', ', ' ')
        hd = hd.replace(',', ' ')
        hd = hd.strip()
        
        if hd.find('TB') >= 0:
            hd = hd[hd.find('TB')-1:hd.find('TB')]
            hd = int(hd) * 1000  # to convert it to GB
            frys_hd_arr.append(int(hd))
        elif hd.find('GB') >= 0:
            hd = hd[0:hd.find('GB')]
            frys_hd_arr.append(int(hd))
    
    
    elif x.find('HD') >= 0:
         # clean data:
        hd = x[x.find('HD')-6:x.find('HD')+2]

        #clean data
        if hd.find('TB') >= 0:
            hd = hd[hd.find('TB')-1:hd.find('TB')]
            hd = int(hd) * 1000 # to convert to GB
            frys_hd_arr.append(int(hd))
        elif hd.find('GB') >=0:
            hd = hd[0:hd.find('GB')]
            frys_hd_arr.append(int(hd))

        else:
            frys_hd_arr.append('')
            
            

    elif x.find('SSD') >= 0:
         # clean data:
        hd = x[x.find('SSD')-6:x.find('SSD')+3]
        
        if hd.find('GB') >=0:
            hd = hd[0:hd.find('GB')]
            frys_hd_arr.append(int(hd))
        else:
            frys_hd_arr.append('')
                        
            

    elif x.find('Storage') >= 0:
         # clean data:
        hd = x[x.find('Storage')-6:x.find('Storage')-1]

        #clean data
        if hd.find('GB') >=0:
            hd = hd[0:hd.find('GB')]
            frys_hd_arr.append(int(hd))
        else:
            frys_hd_arr.append('')
                        
    
    else:
        frys_hd_arr.append('')

frys_hd_arr

  
frys_specs['hd'] = frys_hd_arr

frys_specs  


,0,1,2,3,4,5,6,7,8,screenSize,cpu,ram,hd
0,696897763632,"E6230 12.5""",Laptop With Intel Ci5-3320m Processor,4GB Memory,250GB Hard Drive,Windows 10,None,None,None,12.5,Intel Ci5-3320m,4,250
1,696897764257,"N21 11.6""",Chromebook With Intel Celeron N2840 Processor,4GB Memory,16GB Storage,Chrome OS,None,None,None,11.6,Intel Celeron N2840,4,16
2,696897752902,"TP X220 12.5""",Laptop With Intel Ci5-2520M Processor,4GB Memory,160GB Hard Drive,Windows 10,None,None,None,12.5,Intel Ci5-2520M,4,160
3,696897734717,"8470P 14""",Laptop with Intel Core i5-3230M Processor 8GB ...,"320GB Hard Drive,DVD",Windows 10 **90 Days Warranty**,None,None,None,None,14,Intel Core i5-3230M,8,320
4,696897764264,"TP X240 12.5""",Laptop With Intel Ci5-4300u Processor,8GB Memory,500GB Hard Drive,Windows 10,None,None,None,12.5,Intel Ci5-4300u,8,500
5,724962745983,"E6440 14""",Laptop With Intel Ci5-4200M Processor,16GB Memory,512GB SSD,Windows 10,None,None,None,14,Intel Ci5-4200M,16,512
6,696897743108,"T420 14.1""",Laptop With Ci5-2520M Processor,4GB Memory,250GB Hard Drive and Windows 10,None,None,None,None,14.1,Ci5-2520M,4,250
7,696897751165,"E7440 14""",Touchscreen Laptop With Intel Ci5-4200U Processor,8GB Memory,128GB SSD,Windows 10,None,None,None,14,Intel Ci5-4200U,8,128
8,696897751226,"X131E 11.6""",Laptop With Intel Celeron 1007U Processor,4GB Memory,320GB Hard Drive,Windows 10 Home,None,None,None,11.6,Intel Celeron 1007U,4,320
9,696897750199,"Thinkpad T430 14""",Laptop With Intel Ci5-3210M Processor,8GB Memory,240GB SSD,Windows 10 Pro,None,None,None,14,Intel Ci5-3210M,8,240


## Final Frys Table

In [915]:
frys_specs.head()

final_frys_specs = pd.DataFrame(frys_specs[[0, 'screenSize', 'cpu', 'ram', 'hd']])
final_frys_specs.rename(columns={0: 'upc'}, inplace = True)
final_frys_specs.head()

,upc,screenSize,cpu,ram,hd
0,696897763632,12.5,Intel Ci5-3320m,4,250
1,696897764257,11.6,Intel Celeron N2840,4,16
2,696897752902,12.5,Intel Ci5-2520M,4,160
3,696897734717,14,Intel Core i5-3230M,8,320
4,696897764264,12.5,Intel Ci5-4300u,8,500


In [916]:
frys_df.head()
shorter_frys_df = frys_df[['brand', 'link', 'model', 'price', 'title', 'upc']]
shorter_frys_df.head()

,brand,link,model,price,title,upc
0,Dell,https://www.frys.com/product/9907916?site=sr:S...,E6230i5-3320m,399.0,"Dell E6230 12.5"" Refurbished Laptop With Intel...",696897763632
1,Lenovo,https://www.frys.com/product/9914956?site=sr:S...,N21CeleronN2840,199.0,"Lenovo N21 11.6"" Refurbished Chromebook With I...",696897764257
2,Lenovo,https://www.frys.com/product/9831155?site=sr:S...,TPX220i5-2520M,349.0,"Lenovo TP X220 12.5"" Refurbished Laptop With I...",696897752902
3,HP,https://www.frys.com/product/9647583?site=sr:S...,8470PCi5-3230M,399.0,"HP 8470P 14"" Refurbished Laptop with Intel Cor...",696897734717
4,Lenovo,https://www.frys.com/product/9914966?site=sr:S...,TPX240i5-4300U,399.0,"Lenovo TP X240 12.5"" Refurbished Laptop With I...",696897764264


In [917]:
frys_merged1_df = shorter_frys_df.merge(final_frys_specs, on='upc', how='left')
final_frys_df = frys_merged1_df[['brand', 'model', 'screenSize', 'cpu', 'ram', 'hd', 'upc', 'price', 'link', 'title']]


# Add the load date:
now = DT.datetime.now()

final_frys_df['lastUpdated'] = now.strftime("%Y-%m-%d %H:%M")

final_frys_df.head()

C:\Users\Rika\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,brand,model,screenSize,cpu,ram,hd,upc,price,link,title,lastUpdated
0,Dell,E6230i5-3320m,12.5,Intel Ci5-3320m,4,250,696897763632,399.0,https://www.frys.com/product/9907916?site=sr:S...,"Dell E6230 12.5"" Refurbished Laptop With Intel...",2019-09-24 17:48
1,Lenovo,N21CeleronN2840,11.6,Intel Celeron N2840,4,16,696897764257,199.0,https://www.frys.com/product/9914956?site=sr:S...,"Lenovo N21 11.6"" Refurbished Chromebook With I...",2019-09-24 17:48
2,Lenovo,TPX220i5-2520M,12.5,Intel Ci5-2520M,4,160,696897752902,349.0,https://www.frys.com/product/9831155?site=sr:S...,"Lenovo TP X220 12.5"" Refurbished Laptop With I...",2019-09-24 17:48
3,HP,8470PCi5-3230M,14,Intel Core i5-3230M,8,320,696897734717,399.0,https://www.frys.com/product/9647583?site=sr:S...,"HP 8470P 14"" Refurbished Laptop with Intel Cor...",2019-09-24 17:48
4,Lenovo,TPX240i5-4300U,12.5,Intel Ci5-4300u,8,500,696897764264,399.0,https://www.frys.com/product/9914966?site=sr:S...,"Lenovo TP X240 12.5"" Refurbished Laptop With I...",2019-09-24 17:48


# Load: Loading both Frys and BestBuy data into PostgreSQL

In [918]:
###############################################################
# LOAD: 
###############################################################


# change to your username:password@localhost:5432/laptop_db:
connection = "postgres:helloworld@localhost:5432/laptop_db"
engine = create_engine(f'postgresql://{connection}')

# add the FRYS dataframe to the table:
final_frys_df.to_sql(name="frys_laptops", con=engine, if_exists='replace', index=False)



# add the BEST BUY dataframe to the table:
final_bestbuy_df.to_sql(name="bestbuy_laptops", con=engine, if_exists='replace', index=False)


# VALIDATE TABLE BUILD:
engine.table_names()

['frys_laptops', 'bestbuy_laptops']